#### Import Dependencies

In [ ]:
import tensorflow as tf
import numpy as np
from datetime import datetime
from data_utils import *

#### Load Data

In [ ]:
train_valid_ligids  = np.load('../data/PHARM_TRAIN_X.npy')
train_valid_smiles = np.load('../data/PHARM_TRAIN_SMILES.npy')
train_valid_scores = np.load('../data/Y_train.npy')
test_ligids = np.load('../data/PHARM_TEST_X.npy')
test_smiles = np.load('../data/PHARM_TEST_SMILES.npy')

print('train_valid_ligids shape: {}'.format(train_valid_ligids.shape))
print('train_valid_smiles shape: {}'.format(train_valid_smiles.shape))
print('train_valid_scores shape: {}'.format(train_valid_scores.shape))
print('test_ligids shape: {}'.format(test_ligids.shape))
print('test_smiles shape: {}'.format(test_smiles.shape))

#### Train Validation Test Split

In [ ]:
train_data, validation_data = train_validation_split(train_valid_ligids,
                                                     train_valid_smiles,
                                                     train_valid_scores,
                                                     num_val_lig=3046, 
                                                     num_val_smi=10581)
test_data = Data(test_ligids, 
                 test_smiles, 
                 np.empty(shape=(test_ligids.shape[0]* test_smiles.shape[0]), dtype=np.int8))

del train_valid_ligids, train_valid_smiles, train_valid_scores

#### Define Hyper Parameters

In [ ]:
LEARNING_RATE = .001
LAMBDA = .001
DROPOUT = .5

#### Define Model

In [ ]:
L1_UNITS = 30
L2_UNITS = 30
L3_UNITS = 30
NUM_OUTPUTS = 1

tf.reset_default_graph()

relu = tf.nn.relu
xavier_init = tf.contrib.layers.xavier_initializer()
zero_init = tf.zeros_initializer()
l2_reg = tf.contrib.layers.l2_regularizer(scale=LAMBDA)

with tf.name_scope('inputs') as scope:
    X = tf.placeholder(shape=(None, 176), dtype=tf.float32, name='ligids_smiles')
    Y = tf.placeholder(shape=(None), dtype=tf.float32, name='score')
    training = tf.placeholder_with_default(input=False, shape=(), name='training')
    
with tf.name_scope('hidden_layers') as scope:
    layer1 = tf.layers.dense(inputs=X,
                             units=L1_UNITS, 
                             activation=relu,
                             kernel_initializer=xavier_init,
                             bias_initializer=zero_init,
                             kernel_regularizer=l2_reg,
                             bias_regularizer=l2_reg,
                             name='layer1')
    layer2 = tf.layers.dense(inputs=layer1, 
                             units=L2_UNITS, 
                             activation=relu,
                             kernel_initializer=xavier_init,
                             bias_initializer=zero_init,
                             kernel_regularizer=l2_reg,
                             bias_regularizer=l2_reg,
                             name='layer2')
    layer3 = tf.layers.dense(inputs=layer2, 
                             units=L3_UNITS, 
                             activation=relu,
                             kernel_initializer=xavier_init,
                             bias_initializer=zero_init,
                             kernel_regularizer=l2_reg,
                             bias_regularizer=l2_reg,
                             name='layer3')
    
with tf.name_scope('predicted_score') as scope:
    pred_score = tf.layers.dense(inputs=layer3,
                                 units=NUM_OUTPUTS)
    with tf.get_default_graph().gradient_override_map({"Floor": "Identity"}):
        pred_score = tf.floor(pred_score, name='pred_score')
        
with tf.name_scope('train') as scope:
    mae = tf.losses.absolute_difference(Y,pred_score)
    reg_loss = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    reg_loss = tf.reduce_sum(reg_loss)
    train_loss = tf.losses.mean_squared_error(labels=Y,predictions=pred_score)
    loss = train_loss+reg_loss
    optimizer = tf.train.AdamOptimizer(LEARNING_RATE)
    train_op = optimizer.minimize(loss)

#### Run Model

In [ ]:
# Misc. constants
NUM_EPOCHS = 5
NUM_SAVES_PER_EPOCH = 5
TRAINING_BATCH_SIZE = 10000
num_training_batches = int(train_data.num_scores/TRAINING_BATCH_SIZE)
VALIDATION_BATCH_SIZE = 10000
num_validation_batches = int(validation_data.num_scores/VALIDATION_BATCH_SIZE)
TB_PERIOD = 10

# Saver - Prediction Tensors
tf.get_collection_ref('pred_ops').clear()
tf.get_collection('pred_ops')
tf.add_to_collection('pred_ops', X)
tf.add_to_collection('pred_ops', Y)
tf.add_to_collection('pred_ops', pred_score)
# Saver - Training Tensors
tf.get_collection_ref('train_ops').clear()
tf.get_collection('train_ops')
tf.add_to_collection('train_ops', X)
tf.add_to_collection('train_ops', Y)
tf.add_to_collection('train_ops', loss)
tf.add_to_collection('train_ops', train_op)
saver = tf.train.Saver(max_to_keep=1000)

# Tensorboard - Graph
time_now = datetime.utcnow().strftime('%Y%m%d%H%M%S')
tensorboard_logdir = '../tf_log/run-{}'.format(time_now)
print('tensorboard log_dir: {}'.format(tensorboard_logdir))
writer = tf.summary.FileWriter(tensorboard_logdir)
writer.add_graph(tf.get_default_graph())
# Tensorboard - Summaries
summaries = [tf.summary.scalar('mean_abs_error', mae), 
             tf.summary.scalar('mean_square_error', train_loss), 
             tf.summary.scalar('regularization', reg_loss), 
             tf.summary.scalar('total_loss', loss)]

# Start Session
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(NUM_EPOCHS):       
        # Train Model
        train_data.reset()
        for step in range(num_training_batches):
            ligids_batch, smiles_batch, scores_batch = train_data.next_batch(TRAINING_BATCH_SIZE)
            lig_smi_batch = np.concatenate((ligids_batch,smiles_batch), axis=1)
            _, train_loss = sess.run([train_op, loss], feed_dict={X:lig_smi_batch, Y:scores_batch})
            print('{}/{} train_loss: {}'.format(step, 
                                                num_training_batches, 
                                                train_loss), end='\r')
            # Tensorboard
            if step%TB_PERIOD == 0:
                s = sess.run(summaries, feed_dict={X:lig_smi_batch, Y:scores_batch})
                [writer.add_summary(summary, step) for summary in s]
        # Validation
        validation_data.reset()
        validation_loss, mae_value = [], []
        for step in range(num_validation_batches): 
            # Compute validation loss batch by batch and average
            ligids_batch, smiles_batch, scores_batch = validation_data.next_batch(VALIDATION_BATCH_SIZE)
            lig_smi_batch = np.concatenate((ligids_batch,smiles_batch), axis=1)
            validation_loss_batch, mae_batch = sess.run([loss, mae], 
                                                        feed_dict={X:lig_smi_batch, Y:scores_batch})
            validation_loss.append(validation_loss_batch)
            mae_value.append(mae_batch)
            print('{}/{} validation_loss_batch: {}'.format(step, 
                                                           num_validation_batches, 
                                                           validation_loss_batch), end='\r')
        validation_loss = sum(validation_loss)/len(validation_loss)
        mae_value = sum(mae_value)/len(mae_value)
        print('EPOCH: {:<10} | loss: {:<25} | mae: {:<20}'.format(epoch+1, 
                                                                  validation_loss, 
                                                                  mae_value))
        # Save Model w/ name: e{epoch number}_l{loss}
        saver_filename = 'e{}_l{}'.format(epoch, validation_loss)        
        saver.save(sess, '../models/{}'.format(saver_filename))

#### make_submission.py

In [ ]:
# Batches
TESTING_BATCH_SIZE = 10000
num_test_batches = int(test_data.num_scores/TESTING_BATCH_SIZE)+1

# Loader
saver_filename = 'e8_l2.8511100673734733'
tf.reset_default_graph()
loader = tf.train.import_meta_graph('../models/{}.meta'.format(saver_filename))

with tf.Session() as sess:
    # Load model and tensors
    sess.run(tf.global_variables_initializer())
    loader.restore(sess, '../models/{}'.format(saver_filename))
    X, Y, pred_score = tf.get_collection('pred_ops')

    # Make predictions
    predictions = []
    test_data.reset()
    for batch_num in range(num_test_batches):
        ligids_batch, smiles_batch, empty_batch = test_data.next_batch(TESTING_BATCH_SIZE)
        lig_smi_batch = np.concatenate((ligids_batch,smiles_batch), axis=1)
        prediction_batch = sess.run(pred_score, feed_dict={X:lig_smi_batch, Y:empty_batch})
        predictions.append(prediction_batch)
        batch_bincount = np.bincount(np.squeeze(prediction_batch.astype(int)), minlength=10)
        print('{:<4}/{:<4}: Distribution: {}'.format(batch_num, 
                                 num_test_batches,
                                 batch_bincount), end='\r')
    predictions = np.concatenate(predictions, axis=0)

# Write predictions to submission file
with open('../submissions/submission_{}.csv'.format(saver_filename), 'w') as out_file:
    out_file.write('ligid_pharmid,score\n')
    for glob_index in range(test_data.num_scores):
        out_file.write('{},{}\n'.format(glob_index, int(predictions[glob_index])))